In [16]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import numpy as np

# Load wildfire data
wildfire_df = pd.read_csv("C:/Users/longl/BurnData .csv")

# Handle geometry parsing safely
def safe_wkt_loads(geom_str):
    try:
        return wkt.loads(geom_str)
    except Exception as e:
        print(f"Invalid geometry: {geom_str[:60]}... — {e}")
        return None

wildfire_df['geometry'] = wildfire_df['geometry'].apply(safe_wkt_loads)
wildfire_df = wildfire_df.dropna(subset=['geometry'])

# Convert to GeoDataFrame
wildfire_gdf = gpd.GeoDataFrame(wildfire_df, geometry='geometry', crs="EPSG:4326")

# Add centroid latitude and longitude
wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid
wildfire_gdf['centroid_lat'] = wildfire_gdf['centroid'].y
wildfire_gdf['centroid_lon'] = wildfire_gdf['centroid'].x

# Load airport data
airport_df = pd.read_excel("airports_runways_joined.xlsx")



# Filter airports to only those with an airtanker base
airtanker_df = airport_df[airport_df['airtanker_base'] == True].copy()
print(airtanker_df)
# Ensure no missing coordinates
airtanker_df = airtanker_df.dropna(subset=['latitude_deg', 'longitude_deg'])

# Compute distances to nearest airtanker base
airtanker_results = []
for _, fire in wildfire_gdf.iterrows():
    lat1, lon1 = fire['centroid_lat'], fire['centroid_lon']
    distances = haversine(lat1, lon1, airtanker_df['latitude_deg'], airtanker_df['longitude_deg'])
    
    if len(distances) == 0:
        continue  # No valid airtanker base available

    min_idx = np.argmin(distances)
    nearest_base = airtanker_df.iloc[min_idx]
    
    airtanker_results.append({
        'FIRE ID': fire['unique_id'],
        'FIRE LAT': fire['centroid_lat'],
        'FIRE LON': fire['centroid_lon'],
        'CLOSEST_AIRTANKER_BASE_NAME': nearest_base['name'],
        'AIRPORT IDENT': nearest_base['ident'],
        'RUNWAY LENGTH': nearest_base.get('runway_lengths_ft', None),
        'REGION NAME': nearest_base.get('region_name', None),
        'AIRPORT TYPE': nearest_base.get('type', None),
        'AIRPORT LAT': nearest_base['latitude_deg'],
        'AIRPORT LON': nearest_base['longitude_deg'],
        'DISTANCE_TO_AIRTANKER_BASE': distances.values[min_idx]
    })

# Save results
airtanker_result_df = pd.DataFrame(airtanker_results)
print(airtanker_result_df.head())

# Optionally save to CSV
airtanker_result_df.to_csv("C:/Users/longl/nearest_airtanker_bases_to_fires1.csv", index=False)

Invalid geometry: MULTIPOLYGON (((-122.24913195511729 38.65833332983732, -122.... — ParseException: Expected word but encountered end of stream
Invalid geometry:  -122.36670044747557 38.73333332983061... — ParseException: Expected word but encountered number: '-122.367'
Invalid geometry: MULTIPOLYGON (((-119.24290227413563 37.23749999663116, -119.... — ParseException: Expected number but encountered end of stream
Invalid geometry:  -119.33059466511611 37.32916666328961... — ParseException: Expected word but encountered number: '-119.331'


C:\Users\longl\AppData\Local\Temp\ipykernel_37868\469716364.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid


     ident iata_code icao_code local_code  \
6     K05U       EUE       NaN        05U   
16    K0VG       NaN       NaN        0VG   
107   KABI       ABI      KABI        ABI   
196   KBAM       BAM      KBAM        BAM   
223   KBIH       BIH      KBIH        BIH   
225   KBIL       BIL      KBIL        BIL   
228   KBJC       BJC      KBJC        BJC   
229   KBJI       BJI      KBJI        BJI   
246   KBNO       BNO      KBNO        BNO   
247   KBOI       BOI      KBOI        BOI   
252   KBRD       BRD      KBRD        BRD   
284   KCDC       CDC      KCDC        CDC   
345   KCVH       HLI      KCVH        CVH   
445   KELO       LYU      KELO        ELO   
480   KFAT       FAT      KFAT        FAT   
547   KGEY       GEY      KGEY        GEY   
554   KGIC       IDH      KGIC        GIC   
556   KGJT       GJT      KGJT        GJT   
625   KHLN       HLN      KHLN        HLN   
679   KIGM       IGM      KIGM        IGM   
769   KLGD       LGD      KLGD        LGD   
778   KLKV